# Express Shipment Projection at customer,warehouse and carrier level

In [1]:
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error, SCORERS
from sklearn.model_selection import cross_val_score, KFold

# Splitting data for hyperparameter tuning
from sklearn.model_selection import TimeSeriesSplit
import xgboost as xgb

C:\Users\PSEN\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data Acquring

### We are brining in shipments and order data by warehouse,customers and week in two different tables. SQL queries were used to bring those data from azure data lake

In [5]:
df_shipment = df.copy()
df_order = df1.copy()

# Data pre-processing

In [6]:
df_shipment.dtypes

FiscalWeekLastDate    datetime64[ns]
wh_id                         object
Customer                      object
Customer Name                 object
Units                        float64
dtype: object

In [7]:
df_shipment.head()

,FiscalWeekLastDate,wh_id,Customer,Customer Name,Units
0,2019-08-17,1,3111000,"BEDDING PROS, LLC",5.0
1,2019-08-17,1,3590000-002,DIRECTBUY HOME IMPROVE-,1.0
2,2019-08-17,1,3556100,"JCPENNEY CORP., INC.",22.0
3,2019-08-17,1,9946600-D52,"DSG ILLINOIS, LLC",3.0
4,2019-08-17,1,1886800-07,GIBSON MCDONALD,21.0


In [8]:
df_shipment = df_shipment.dropna().reset_index(drop=True)

In [9]:
df_shipment['customer'] = df_shipment['Customer Name'].apply(lambda x: 'AMAZON' if 'AMAZON' in x else
                                                             ('WAYFAIR' if 'WAYFAIR' in x else
                                                              ('WALMART' if 'WALMART' in x else
                                                               ('ASHCOMM' if 'ASHCOMM' in x else 'OTHER'))))

In [10]:
df_shipment = df_shipment.rename(
    columns={'FiscalWeekLastDate': 'Date', 'Units': 'Shipped Unit'})
df_shipment.drop(['Customer', 'Customer Name'], axis=1, inplace=True)
df_shipment.head()

,Date,wh_id,Shipped Unit,customer
0,2019-08-17,1,5.0,OTHER
1,2019-08-17,1,1.0,OTHER
2,2019-08-17,1,22.0,OTHER
3,2019-08-17,1,3.0,OTHER
4,2019-08-17,1,21.0,OTHER


In [11]:
df_order.dtypes

wh_id                    object
Date             datetime64[ns]
Customer                 object
Customer Name            object
Total order             float64
dtype: object

In [12]:
df_order = df_order.dropna().reset_index(drop=True)
df_order['customer'] = df_order['Customer Name'].apply(lambda x: 'AMAZON' if 'AMAZON' in x else
                                                       ('WAYFAIR' if 'WAYFAIR' in x else
                                                        ('WALMART' if 'WALMART' in x else
                                                         ('ASHCOMM' if 'ASHCOMM' in x else 'OTHER'))))

In [13]:
# df_order['Date']=pd.to_datetime(df_order['Date'])

df_order.head()

,wh_id,Date,Customer,Customer Name,Total order,customer
0,28,2019-06-15,4444400-100,ASHCOMM LLC,169.0,ASHCOMM
1,28,2019-06-15,2921200-01,ARMY & AIR FORCE,1.0,OTHER
2,28,2019-06-15,3220800,WYCKES FURNITURE,9.0,OTHER
3,28,2019-06-15,3061700-11,PAYLESS FURNITURE INC.,8.0,OTHER
4,28,2019-06-15,1682300-01,NIX HOME CENTER,11.0,OTHER


In [14]:
df_shipment = df_shipment.groupby(['wh_id', 'Date', 'customer'], as_index=False)[
    'Shipped Unit'].sum()
df_order = df_order.groupby(['wh_id', 'Date', 'customer'], as_index=False)[
    'Total order'].sum()

In [15]:
df_final = pd.merge(df_shipment, df_order, how='left',
                    on=['Date', 'wh_id', 'customer'])

In [16]:
df_final.head()

,wh_id,Date,customer,Shipped Unit,Total order
0,1,2019-08-17,AMAZON,457.0,579.0
1,1,2019-08-17,ASHCOMM,1634.0,2000.0
2,1,2019-08-17,OTHER,679.0,816.0
3,1,2019-08-17,WALMART,71.0,86.0
4,1,2019-08-17,WAYFAIR,733.0,791.0


In [17]:
df_final = df_final.dropna().reset_index(drop=True)

In [18]:
df_data = pd.concat([df_final, pd.get_dummies(
    df_final[['wh_id', 'customer']])], axis=1)
df_data.drop(['wh_id', 'customer'], axis=1, inplace=True)

In [19]:
df_data.head()

,Date,Shipped Unit,Total order,wh_id_1,wh_id_15,wh_id_16,wh_id_17,wh_id_28,wh_id_42,wh_id_5,wh_id_ECR,customer_AMAZON,customer_ASHCOMM,customer_OTHER,customer_WALMART,customer_WAYFAIR
0,2019-08-17,457.0,579.0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,2019-08-17,1634.0,2000.0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,2019-08-17,679.0,816.0,1,0,0,0,0,0,0,0,0,0,1,0,0
3,2019-08-17,71.0,86.0,1,0,0,0,0,0,0,0,0,0,0,1,0
4,2019-08-17,733.0,791.0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [20]:
df_data.sort_values(['Date'],inplace=True,ignore_index=True)

### To make the model understand the covid 19 period, we are creating a covid 19 variable that will indicate the period when the sales were down when covid 19 hit.

In [21]:
# covid variable
# sales spike during covid was noticed between March 21st to June 27 of 2020.
covid_19 = ['2020-03-21', '2020-06-27']
covid_19 = pd.to_datetime(covid_19)

df_data['covid_19'] = df_data['Date'].apply(
    lambda x: 1 if x >= covid_19[0] and x <= covid_19[1] else 0)


In [22]:
df_data['Week'] = df_data['Date'].dt.week
df_data['Month'] = df_data['Date'].dt.month
df_data['Year'] = df_data['Date'].dt.year
df_data = df_data.drop('Date', axis=1)
df_data.head()

C:\Users\PSEN\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


,Shipped Unit,Total order,wh_id_1,wh_id_15,wh_id_16,wh_id_17,wh_id_28,wh_id_42,wh_id_5,wh_id_ECR,customer_AMAZON,customer_ASHCOMM,customer_OTHER,customer_WALMART,customer_WAYFAIR,covid_19,Week,Month,Year
0,457.0,579.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,33,8,2019
1,1634.0,2000.0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,33,8,2019
2,679.0,816.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,33,8,2019
3,71.0,86.0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,33,8,2019
4,733.0,791.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,33,8,2019


### Way day and prime day are the two days when we need to ship more units to wayfair and amazon. That is why we are making two variables to mention wayday and primeday

In [23]:
way_day=pd.DataFrame(['2019-04-10', '2019-04-11','2020-09-23', '2020-09-24','2021-04-28', '2021-04-29'],columns=['date'])
prime_day=pd.DataFrame(['2019-07-15', '2019-07-16','2020-10-13', '2020-10-14','2021-06-21', '2021-06-22'],columns=['date'])

# convert to datetime
way_day['date'] = pd.to_datetime(way_day['date'])
prime_day['date'] = pd.to_datetime(prime_day['date'])



def wayday(week,year,customer):
    way=0
    if str(week)+"_"+str(year) in ((way_day['date'].dt.week).astype(str) 
                                   + "_" + (way_day['date'].dt.year).astype(str)).tolist() and customer==1:
        way=1
    else:
        way=0
    return way



def primeday(week,year,customer):
    prime=0
    if str(week)+"_"+str(year) in ((prime_day['date'].dt.week).astype(str) 
                                   + "_" + (prime_day['date'].dt.year).astype(str)).tolist() and customer==1:
        prime=1
    else:
        prime=0
    return prime

df_data['Wayday']=df_data[['Week','Year','customer_WAYFAIR']].apply(lambda x: wayday(*x),axis=1)
#df_data['Wayday_v1']=df_data[['Week','Year','customer_WAYFAIR']].apply(lambda x: wayday(*x),axis=1)
df_data['Primeday']=df_data[['Week','Year','customer_AMAZON']].apply(lambda x: primeday(*x),axis=1)
#df_data['Primeday_v1']=df_data[['Week','Year','customer_AMAZON']].apply(lambda x: primeday_v1(*x),axis=1)



C:\Users\PSEN\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  del sys.path[0]
C:\Users\PSEN\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [24]:
df_data

,Shipped Unit,Total order,wh_id_1,wh_id_15,wh_id_16,wh_id_17,wh_id_28,wh_id_42,wh_id_5,wh_id_ECR,...,customer_ASHCOMM,customer_OTHER,customer_WALMART,customer_WAYFAIR,covid_19,Week,Month,Year,Wayday,Primeday
0,457.0,579.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,33,8,2019,0,0
1,1634.0,2000.0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,33,8,2019,0,0
2,679.0,816.0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,33,8,2019,0,0
3,71.0,86.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,33,8,2019,0,0
4,733.0,791.0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,33,8,2019,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2932,838.0,862.0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,24,6,2021,0,0
2933,227.0,168.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,24,6,2021,0,0
2934,1213.0,1252.0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,24,6,2021,0,0
2935,1009.0,1025.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,24,6,2021,0,0


# Model Building

In [25]:
y = df_data['Shipped Unit']
x = df_data.drop(['Shipped Unit'], axis=1)

b = int(len(df_data)*.80)
x_train, x_test = x.iloc[0:b, ], x.iloc[b:]
y_train, y_test = y.iloc[0:b], y.iloc[b:]

In [26]:
x.dtypes

Total order         float64
wh_id_1               uint8
wh_id_15              uint8
wh_id_16              uint8
wh_id_17              uint8
wh_id_28              uint8
wh_id_42              uint8
wh_id_5               uint8
wh_id_ECR             uint8
customer_AMAZON       uint8
customer_ASHCOMM      uint8
customer_OTHER        uint8
customer_WALMART      uint8
customer_WAYFAIR      uint8
covid_19              int64
Week                  int64
Month                 int64
Year                  int64
Wayday                int64
Primeday              int64
dtype: object

In [27]:
rf_model = RandomForestRegressor(random_state=0)
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')
# rf_model.fit(x_train,y_train)

In [28]:
tss = TimeSeriesSplit(n_splits=3)

In [29]:
rf_scores = cross_val_score(
    rf_model, X=x, y=y, cv=tss, scoring='neg_mean_absolute_percentage_error')
xgb_scores = cross_val_score(
    xgb_model, X=x, y=y, cv=tss, scoring='neg_mean_absolute_percentage_error')

In [30]:
print(
    f'The corss validation scores for Random Forest Regression are {abs(rf_scores)}')
print(
    f'The cross validation scores for xgboost regression are {abs(xgb_scores)}')

The corss validation scores for Random Forest Regression are [0.09394811 0.46376074 0.14528869]
The cross validation scores for xgboost regression are [0.10726268 0.5096494  0.3299462 ]


### On 3 fold cross-validation, Random Forest Regression is doing better than xgboost. We will use Random Forest by tuning it's parameter

# Tuning Hyperparameter

In [31]:
# functions to train random forest regression model
def model_rf(x_train, y_train):  # Function fits the random forest regression
    model = RandomForestRegressor(random_state=0)
    return model.fit(x_train, y_train)


model = model_rf(x_train, y_train)

In [32]:
# Funtion takes a fitted model and predicts based on x_test values and calculates the MAPE
def result(model, x_test, y_true):
    y_pred = model.predict(x_test)
    mape = mean_absolute_percentage_error(y_true, y_pred)*100
    return mape

### MAPE without tuning hyper-parameter

In [33]:
result(model, x_test, y_test)

10.900985336410942

In [34]:
from sklearn.model_selection import GridSearchCV

# define the grid
params = {
    "n_estimators": [50, 100, 150],
    "max_features": [0.33, 0.66, 1.0],
    "min_samples_split": [2, 8, 14],
    "min_samples_leaf": [1, 5, 10, 15],
    "bootstrap": [True, False]
}

# setup the grid search
grid_search = GridSearchCV(model,
                           param_grid=params,
                           cv=tss,
                           verbose=1,
                           n_jobs=4,
                           return_train_score=True)

rf_grid = grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


### MAPE after tuning hyper-parameter

In [35]:
result(rf_grid, x_test, y_test)

10.702971895157017

# Check with auto ml library (TPOT)

### To understand what pipeline can fit our data well, we tried TPOT auto ml package to test pipelines

In [37]:
from tpot import TPOTRegressor

C:\Users\PSEN\AppData\Local\Continuum\anaconda3\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [38]:
tpot = TPOTRegressor(generations=15, population_size=20, verbosity=2,
                     cv=tss, n_jobs=1, scoring='neg_mean_absolute_percentage_error')
tpot.fit(x_train, y_train)
print(tpot.score(x_test, y_test))
tpot.export('tpot_optimum_pipeline.py')

Optimization Progress:   0%|          | 0/320 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -0.18362999084606613

Generation 2 - Current best internal CV score: -0.18362999084606613

Generation 3 - Current best internal CV score: -0.18362999084606613

Generation 4 - Current best internal CV score: -0.18362999084606613

Generation 5 - Current best internal CV score: -0.182032964106323

Generation 6 - Current best internal CV score: -0.18105942971205616

Generation 7 - Current best internal CV score: -0.18036160412950597

Generation 8 - Current best internal CV score: -0.17993231443960025

Generation 9 - Current best internal CV score: -0.1789896938773851

Generation 10 - Current best internal CV score: -0.1789896938773851

Generation 11 - Current best internal CV score: -0.1789896938773851

Generation 12 - Current best internal CV score: -0.1789896938773851

Generation 13 - Current best internal CV score: -0.1789896938773851

Generation 14 - Current best internal CV score: -0.1789896938773851

Generation 15 - Current best interna

### We can see that on training data TPOT is giving us app. 18% MAPE. Finally in the test data, it gave us a 9.81% MAPE. In comparison with our Random Forest, TOPT is giving us better results both on training and test set